In [11]:
from docx import *
import copy

## Helper Functions

In [12]:
def safe_point(doc_obj):
   if doc_obj is not None:
      return doc_obj.pt
   else: return None
   
def safe_tab(doc_obj):
   if doc_obj is not None:
      return abs(doc_obj)/91440
   else: return None

def batch_dict_update(dict, new_items_dict = 0, zipped_items = 0):
   if zipped_items != 0:
      raise NotImplementedError
   elif new_items_dict != 0:
      for k, v in new_items_dict.items():
         dict[k] = v
      return dict
         # print(dict)
   else:
      raise RuntimeError

#    #how to handle alphebetical groupings
#       #remove any that have 18pt size
#    #check all runs for anything bold that ISN'T a run[0]
#    if indents_bool and runs_bool_strict:
#       is_root = True
#       is_certain = True
#    elif indents_bool or runs_bool_lax:
#       is_root = True
#       is_certain = False
#    else:
#       is_root = False
#       is_certain = False  
#    # print({'is_root':is_root, 'is_certain':is_certain},(indents_bool,runs_bool_strict,runs_bool_lax),'\nend\n')
#    return (is_root, is_certain)


def check_for_targetted_indents(para, config = None):
   """
   Takes a paragraph sub dict and the attendent runs dict. 
   Returns: tuple   (indent_one, indent_0, neither)
         #OLD#dictionary with two boolean values keyed to: 'is_root' and 'is_certain'
   Note: Config functionality not implemented yet, or all that well prepped
   """
   # print("\npara_____________",para)
   # print(runs_dict)
   if config is not None:
      raise NotImplementedError
   else:
      config = {'lower_root_para_features': {
                                 # 'first_line_indent': 1, 
                                 # 'left_indent': 2, 
                                 'para_indent': 1,
                                 },
               'upper_root_para_features': {
                                 # 'first_line_indent': 2, 
                                 # 'left_indent': 2, 
                                 'para_indent': 0,
                                 }}
   bool_single_indent = [para['para_features'][k] == v for k,v in config['lower_root_para_features'].items()]
   if all(bool_single_indent):
      return (True, False, False) #indent_one, indent_0, neither
   bool_no_indent = [para['para_features'][k] == v for k,v in config['upper_root_para_features'].items()]
   if all(bool_no_indent):
      return (False, True, False) #indent_one, indent_0, neither
   
   return (False, False, True) #indent_one, indent_0, neither




def check_run_specific_font_features(runs_dict_entry, run_num, config = None):
   """
   Takes a paragraph sub dict and the attendent runs dict. 
   Returns: (bool, bool, bool, bool) #upper, mid, bold, reject
   Note: Config functionality not implemented yet, or all that well prepped
   """
   # print("\npara_____________",para)
   # print(runs_dict)
   if config is not None:
      raise NotImplementedError
   else:
      config = {
               'lower_root_runs_features': {'run_bold': True},

               'reject_runs_features': {'run_size_pt': 18.0},
               'upper_root_runs_features': {'run_size_pt': 12.0}}
   # print(runs_dict_entry)
   # print(runs_dict_entry['run_features'])
   reject_run = [runs_dict_entry['run_features'][k] == v for k,v in config['reject_runs_features'].items()]
   if all(reject_run):
      return (False, False, False, True) #upper, mid, bold, reject
   else: pass

   bool_bold = [runs_dict_entry['run_features'][k] == v for k,v in config['lower_root_runs_features'].items()]
   if all(bool_bold):
      return (False, False, True, False) #upper, mid, bold, reject
   else: pass

   if run_num == 0:
      try:
         begins_with_hyphen = runs_dict_entry['run_text'][0] is '-'
      except:
         return (False, False, False, False) #upper, mid, bold, reject
      upper_root_runs_features = [runs_dict_entry['run_features'][k] == v for k,v in config['upper_root_runs_features'].items()]
      if all(upper_root_runs_features): 
         if begins_with_hyphen: return (False, True, False, False) #upper, mid, bold, reject
         else: return (True, False, False, False) #upper, mid, bold, reject
      else:
         text = runs_dict_entry['run_text'].split(" ")[0]
         if text == text.upper() and text != text.lower():
            if begins_with_hyphen: return (False, True, False, False) #upper, mid, bold, reject
            else: return (True, False, False, False) #upper, mid, bold, reject
   
   return (False, False, False, False) #upper, mid, bold, reject
   

                                       #how to handle alphebetical groupings
                                          #remove any that have 18pt size
                                       #check all runs for anything bold that ISN'T a run[0]
                           #how to handle non_indented
                              #look just for the size
                              #check all run[0].text and see if text.UPPER == text
                           #???how to handle non_indented not right font size
                              #check all run[0].text and see if text.UPPER == text
                              #check if run[0].strip().split(' ')[0] is the same UPPER and plain.
                              #have doublecheck of no helv?
  




## python-docx extraction & paragraph level features

In [13]:
filename = "Fula_Dictionary-repaired.docx"
# filename = "pasted_docx page 1.docx"
document = Document(filename)

do_save_file = False
out_name = "simplified_parsed_dict_COMPLETE_v1.txt"


stop = 0


paras_dict = {}
pdict = {}
for para in document.paragraphs:

   pdict["first_line_indent"] = int(safe_tab(para.paragraph_format.first_line_indent) or 0)


   pdict["left_indent"] = int(safe_tab(para.paragraph_format.left_indent) or 0)

   pdict["para_indent"] = pdict["left_indent"]-pdict["first_line_indent"]
   tabs = "\t"*pdict["para_indent"]
   # print(pdict["para_indent"])

   paras_dict[stop] = {
                     # "para":para,
                     "runs": [run for run in para.runs], 
                     "pretty_plain_text": f"{tabs} {para.text}",
                     "para_features": pdict.copy(), 
                     # "first_line_indent": pdict["first_line_indent"],
                     # "left_indent": pdict["left_indent"],
                     # "para_indent": pdict["para_indent"]
                      }
   stop +=1
   # if stop >=60:
      # break
del document

## Iterate over DOCX with dict() Extractor

In [14]:
# from deepdiff import DeepDiff
upper_roots_only_dict = {}
inner_roots_only = {}
rooted_dict = {}


#parent root
   #create program to test for root
      #> assert ('first_line_indent': 2, 'left_indent': 2, 'para_indent': 0},)
      #> 'run_size_pt': 12.0
   #validation
      #only one of those are true
      #test #> str.strip() == str.strip().upper()
   #add root to root dict ......., and nest all following paras under it
      #validate
         #save count of children

#sub-root
   #program for subroot
      #> assert ('first_line_indent': 1, 'left_indent': 2, 'para_indent': 1},)
      #> one of the runs after this contains
         #> 'run_font_name': 'TmsRmn 10pt', 'run_bold': True
   #> validate structure
      #> this is always run 0?
      #> both of the above always co-occur.
for p in paras_dict.keys():
   runs = paras_dict[p]["runs"]
   i = 0
   d = {}
   dupes = []
   prev = None
   is_upper_root, is_upper_root_certain, is_inner_root, is_inner_root_certain = False, False, False, False
   for run in runs:

      features = {
            "run_font_name": run.font.name,
            "run_bold": run.bold,
            "run_italic": run.italic,
            "run_size_pt": safe_point(run.font.size),
            }

      same_as_prev = features == prev
      d[i] = {}
      d[i]["run_text"] = run.text

      if same_as_prev:
         dupes.append(i)
         d[i]["same_format_as_prev"] = True
         look_back_num_index = 1
         looking_at_a_dup = True

         while looking_at_a_dup:
            try:
               previous_run = d[i-look_back_num_index]
               if previous_run.get("same_format_as_prev",False):
                  look_back_num_index += 1
               else:
                  looking_at_a_dup = False
            except:
               break

         prev_index = i - look_back_num_index
         prev_text = previous_run["run_text"]

         new_text = prev_text + run.text
         d[prev_index]["run_text"] = new_text
         
      else:
         # d[i]["diffs_from_prev"] = DeepDiff(prev, features)
         prev = features.copy()
         d[i]['run_features'] = features
         d[i]['para_all_text'] = paras_dict[p]['pretty_plain_text']
         d[i]['para_features'] = paras_dict[p]['para_features']
         # d[i] = batch_dict_update(d[i],features)

      i += 1
   # new
   # text_from_runs = []
   indent_1, indent_0, neither_indent = check_for_targetted_indents(paras_dict[p])
   indexes_for_all_runs = d.copy().keys()
   

   for k in indexes_for_all_runs:
      if k in dupes:
         d.pop(k)
      else: 
         # pass
         # text_from_runs.append(d[k]['run_text'].strip())
         d[k]['run_text'] = d[k]['run_text'].strip()

         empty_root = not bool(rooted_dict)
         
         upper_root, middle_root, bold, reject_run = check_run_specific_font_features(d[k],k)
         # prev_inner_root = None
         if upper_root:
            d['upper_root'] = k #Note, can only be found on Run 0
            ###
            upper_roots_only_dict[p] = copy.deepcopy(d[k])
            rooted_dict[d[k]['run_text']] = {'index':(p,k), 
                                             'subordinate_content': {(p,k):d[k]}, #{p:d,p_n:d_n, '...':'...'},
                                             'subordinate_root':{},                
                                             'stats':{
                                                      'middle_root_count': 0,
                                                      'direct_inner_root_count': 0,
                                                      }
                                             }
            prev_upper_root = rooted_dict[d[k]['run_text']]
            prev_upper_middle_root = rooted_dict[d[k]['run_text']]
            prev_any_root = rooted_dict[d[k]['run_text']]
         elif middle_root:
            d['middle_root'] = k #Note, can only be found on Run 0
            # upper_roots_only_dict[p] = copy.deepcopy(d[k])
            prev_upper_root['subordinate_root'][d[k]['run_text']] = {
                                                'index':(p,k), 
                                                'subordinate_content': {(p,k):d[k]}, #{p:d,p_n:d_n, '...':'...'},
                                                'subordinate_root':{},                 
                                                'stats':{
                                                      # 'middle_root_count': 0,
                                                      'direct_inner_root_count': 0,
                                                      }
                                                }
            prev_upper_root['stats']['middle_root_count'] += 1
            
            # prev_upper_root = rooted_dict[rooted_dict[d[k]['run_text']]]
            prev_upper_middle_root = prev_upper_root['subordinate_root'][d[k]['run_text']]
            prev_any_root = prev_upper_root['subordinate_root'][d[k]['run_text']]
         elif bold:
            d['bold'] = k #Note, can be found on any run
            prev_upper_middle_root['subordinate_root'][d[k]['run_text']] = {
                                                'index':(p,k), 
                                                'subordinate_content': {(p,k):d[k]}, #{p:d,p_n:d_n, '...':'...'},
                                                # 'subordinate_root':{},                 
                                                # 'stats':{
                                                         # 'middle_root_count': 0,
                                                         # 'inner_root_count': 0,
                                                         # }
                                                }
            
            prev_upper_root['stats']['direct_inner_root_count'] += 1
            # prev_upper_root = rooted_dict[prev_upper_root_key]['subordinate_root'][d[k]['run_text']]
            # prev_upper_middle_root = prev_upper_root['subordinate_root'][d[k]['run_text']]
            prev_any_root = prev_upper_middle_root['subordinate_root'][d[k]['run_text']]
            prev_inner_root_key = d[k]['run_text']
            prev_inner_root_index = prev_any_root['index']
            inner_roots_only[prev_inner_root_key] = copy.deepcopy(prev_any_root)
         elif reject_run:
            d.pop(k) 
         else:  #subordinate content of the last root
            if empty_root:
               continue
            else: 
               prev_any_root['subordinate_content'][(p,k)] = d[k]
               if prev_inner_root_index == prev_any_root['index']:
                  inner_roots_only[prev_inner_root_key]['subordinate_content'][(p,k)] = d[k]

   
   paras_dict[p]["runs"] = d



In [15]:

if do_save_file:
      import json       
      with open(out_name,"w", encoding="utf-8") as data:  
            data.write(str(paras_dict))

In [16]:
# readable_dict = {}
# for k in paras_dict.keys():
#    readable_dict[k] = {"runs": k["runs"],
#    }
inner_roots_only
# rooted_dict

{'a': {'index': (5, 0),
  'subordinate_content': {(5, 0): {'run_text': 'a',
    'run_features': {'run_font_name': 'TmsRmn 10pt',
     'run_bold': True,
     'run_italic': None,
     'run_size_pt': None},
    'para_all_text': '\t a  prn,sbj,sf  DFZH  Z<->',
    'para_features': {'first_line_indent': 1,
     'left_indent': 2,
     'para_indent': 1}},
   (5, 1): {'run_text': 'prn,sbj,sf',
    'run_features': {'run_font_name': 'TmsRmn 10pt',
     'run_bold': None,
     'run_italic': True,
     'run_size_pt': None},
    'para_all_text': '\t a  prn,sbj,sf  DFZH  Z<->',
    'para_features': {'first_line_indent': 1,
     'left_indent': 2,
     'para_indent': 1}},
   (5, 2): {'run_text': 'DFZH  Z<->',
    'run_features': {'run_font_name': 'Helv 8pt',
     'run_bold': None,
     'run_italic': None,
     'run_size_pt': 8.0},
    'para_all_text': '\t a  prn,sbj,sf  DFZH  Z<->',
    'para_features': {'first_line_indent': 1,
     'left_indent': 2,
     'para_indent': 1}},
   (6, 0): {'run_text': 'yo